<a href="https://colab.research.google.com/github/Doclikam/Machine-Learning-2/blob/main/week%203/time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Beijing-Multi-Site-Air-Quality-Data-Data-Set**

# **Background**

The air pollution crisis in Beijing, China, during 2013 has raised significant concerns regarding public health and environmental sustainability.
In response to this crisis, there is a need to conduct a comprehensive time-series analysis to understand the temporal patterns and trends of air pollution levels, identify key factors influencing air quality, and develop predictive models for forecasting future pollution levels.

# **objectives**
Task 1: Data import, cleanup, and preprocessing

Task 2: Exploratory data analysis

Task 3: Predictive modeling

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import pandas as pd


In [2]:
df=pd.read_csv('/content/dataset.csv')

In [13]:
df.head()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,4,4,4,7,300,77,-0.7,1023,-18.8,0,NNW,4.4,Aotizhongxin
1,2,2013,3,1,1,8,8,4,7,300,77,-1.1,1023.2,-18.2,0,N,4.7,Aotizhongxin
2,3,2013,3,1,2,7,7,5,10,300,73,-1.1,1023.5,-18.2,0,NNW,5.6,Aotizhongxin
3,4,2013,3,1,3,6,6,11,11,300,72,-1.4,1024.5,-19.4,0,NW,3.1,Aotizhongxin
4,5,2013,3,1,4,3,3,12,12,300,72,-2,1025.2,-19.5,0,N,2,Aotizhongxin


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13860 entries, 0 to 13859
Data columns (total 18 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   No       13860 non-null  int64  
 1   year     13860 non-null  int64  
 2   month    13860 non-null  int64  
 3   day      13860 non-null  int64  
 4   hour     13860 non-null  int64  
 5   PM2.5    13860 non-null  object 
 6   PM10     13860 non-null  object 
 7   SO2      13860 non-null  object 
 8   NO2      13860 non-null  object 
 9   CO       13860 non-null  object 
 10  O3       13860 non-null  object 
 11  TEMP     13860 non-null  float64
 12  PRES     13860 non-null  float64
 13  DEWP     13859 non-null  float64
 14  RAIN     13859 non-null  float64
 15  wd       13859 non-null  object 
 16  WSPM     13859 non-null  float64
 17  station  13859 non-null  object 
dtypes: float64(5), int64(5), object(8)
memory usage: 1.9+ MB


In [4]:
#drop non-consequential columns
df=df.drop(columns='No')

In [5]:
#Rename the columns
df.rename(columns={'wd':'wind_direction','DEWP':'dew_point_temp','TEMP':'temp','RAIN':'rain','WSPM':'wind_speed','PRES':'pressure'},inplace=True)


In [6]:
#coverting the year, month, day, hour to the date column
df['date']=pd.to_datetime(df[['year', 'month', 'day', 'hour']])
df.set_index('date', inplace=True)
df.drop(columns=(['year', 'month', 'day', 'hour']), inplace=True)
df.head(3)

,PM2.5,PM10,SO2,NO2,CO,O3,temp,pressure,dew_point_temp,rain,wind_direction,wind_speed,station
date,,,,,,,,,,,,,
2013-03-01 00:00:00,4,4,4,7,300,77,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin
2013-03-01 01:00:00,8,8,4,7,300,77,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin
2013-03-01 02:00:00,7,7,5,10,300,73,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin


In [12]:
#These columns are objects(PM2.5	PM10	SO2	NO2	CO	O3)
object_cols=['PM2.5', 'PM10',	'SO2', 'NO2', 'CO',	'O3']

for col in object_cols:
  # Replace '?' with NaN to handle missing values
  df[object_cols] = df[object_cols].replace('?', np.nan)
  #converting them to numerical columns
  df[col].astype(float)

In [14]:
#missingness of the data
df.isnull().sum().sort_values(ascending=False)

,0
CO,1014
O3,747
SO2,238
NO2,217
PM2.5,128
PM10,115


It appears that the columns 'PM2.5', 'PM10',	'SO2', 'NO2', 'CO',	'O3' have missing values. as such we interpolate the data to fill this missingness.

# **Exploratory Data Analysis**

In [10]:
df['PM2.5'].plot()

TypeError: no numeric data to plot